In [119]:
import folium
import pandas as pd
import numpy as np
import json
import requests
from bs4 import BeautifulSoup
from lxml import etree
import requests
from IPython.display import display, HTML
import re
import matplotlib.pyplot as plt
import time
import pickle
import os

In [120]:
API_KEY = 'RGAPI-13b30cdc-783c-47f6-85f0-d43855b5e322'
SEASON = '2015'
RANKED_QUEUE = 'RANKED_SOLO_5x5'
TEN_SECONDS_MAX = '9'
TEN_MINUTES_MAX = '490'
MIN_MATCHES = 50
matchlist = []
player_list = []

In [121]:
def getUser(summoner_name_list):
    summoner_names = []
    for summoner_name in summoner_name_list:
        url = 'https://euw.api.pvp.net/api/lol/euw/v1.4/summoner/by-name/' + summoner_name + '?api_key='+API_KEY
        info = requests.request("GET", url)
        summoner_names.append(info.json()[summoner_name]['id'])
    return summoner_names

def getStats(summoner_id):
    url = 'https://euw.api.pvp.net/api/lol/euw/v1.3/stats/by-summoner/' + summoner_id + '/ranked?season=SEASON' + SEASON + '&api_key='+API_KEY
    info = requests.request("GET", url)
    return info.json()

In [135]:
def getMatchList(summoner_id_list):
    matchlist = []
    for summoner_id in summoner_id_list:
        url = url = 'https://euw.api.pvp.net/api/lol/euw/v2.2/matchlist/by-summoner/' + str(summoner_id) +'?rankedQueues=' + RANKED_QUEUE + '&seasons=SEASON' + SEASON + '&api_key='+API_KEY
        summoner_matchlist = requests.request("GET", url).json()
        matchlist += [i['matchId'] for i in summoner_matchlist['matches'] if i['matchId'] not in matchlist]
    return matchlist

def getPlayersList(matchlist):
    time_ten_seconds = time.time()
    time_ten_minutes = time.time()
    players_list = []
    print(len(matchlist))
    for match_id in matchlist:
        url = 'https://euw.api.pvp.net/api/lol/euw/v2.2/match/' + str(match_id) + '?api_key=' + API_KEY
        info = requests.request("GET", url)
        headers = info.headers
        body = info.json()
        if len(body.keys()) == 14:
            players_match=[body['participantIdentities'][i]['player']['summonerId'] for i in range(len(body['participantIdentities'])) if i not in players_list]
            players_list += players_match
            time_ten_seconds, time_ten_minutes = checkTime(headers, time_ten_seconds, time_ten_minutes)
        
    return players_list

In [123]:
def checkTime(headers, time_ten_seconds, time_ten_minutes):
    ten_seconds_limit = headers['X-Rate-Limit-Count'].split(',')[0].split(':')[0]
    ten_minutes_limit = headers['X-Rate-Limit-Count'].split(',')[1].split(':')[0]
    if ten_seconds_limit == '1':
        time_ten_seconds = time.time()
    if ten_minutes_limit == '1':
        time_ten_minutes = time.time()
    elif ten_minutes_limit == TEN_MINUTES_MAX:
        to_sleep = 600 - (time.time()-time_ten_minutes)
        if to_sleep < 0:
            to_sleep = 0
        print("Ten minutes limit! Sleeping " + str(to_sleep) + " seconds")
        time.sleep(to_sleep)
        time_ten_minutes = time.time()
    elif ten_seconds_limit == TEN_SECONDS_MAX:
        to_sleep = 10 - (time.time() - time_ten_seconds)
        if to_sleep < 0:
            to_sleep = 0
        #print("Ten seconds limit! Sleeping " + str(to_sleep) + " seconds")
        time.sleep(to_sleep)
        time_ten_seconds = time.time()
    return time_ten_seconds, time_ten_minutes

In [35]:
list_name_seeds_summoners = ["makiaveliko84",          #always Bronze
                        "limoneslocos",           #Silver -> Silver
                        "skinny1love",          #Gold015 -> Platinum16
                        "singedairlines",        #Platinum -> Platinum
                        "bestluxitaxd"]        #Diamond  -> Diamond

list_ids_seeds_summoners= getUser(list_name_seeds_summoners)

In [140]:
list_ids_seeds_summoners = ['31970701']

In [141]:
for i in range(1,len(list_ids_seeds_summoners)):
    matchlist = getMatchList([list_ids_seeds_summoners[i]])
    player_list = getPlayersList(matchlist)
    print("Loading pickle " + str(i))
    with open('./summoner_ids.pkl', 'rb') as f:
        saved = pickle.load(f)
        saved['ids'] += player_list
    print("Saving player " + str(i))
    with open('./summoner_ids.pkl', 'wb') as f:
        pickle.dump(saved, f)
print(len(saved['ids']))

32088


In [68]:
if not os.path.isfile('./summoner_ids.pkl'):
    info = {'ids': player_list }
    with open('./summoner_ids.pkl', 'wb') as f:
        pickle.dump(info, f)
if not os.path.isfile('./summoner_stats.pkl'):
    info = {}
    with open('./summoner_stats.pkl', 'wb') as f:
        pickle.dump(info, f)

In [143]:
with open('./summoner_ids.pkl', 'rb') as f:
    ids_json = pickle.load(f)
    ids_list = ids_json['ids']
with open('./summoner_stats.pkl', 'rb') as f:
    stats_json = pickle.load(f)

In [166]:
time_ten_seconds = time.time()
time_ten_minutes = time.time()
counter = 1
for summoner_id in ids_list:
    id_ = str(summoner_id)
    if id_ in stats_json:
        counter += 1
        if counter%100 ==0:
            print(str(counter), "players done")
        continue
    if counter%100 ==0:
        with open('./summoner_stats.pkl', 'wb') as f:
            pickle.dump(stats_json, f)
        print(str(counter), "players done")

    url = 'https://euw.api.pvp.net/api/lol/euw/v1.3/stats/by-summoner/' + id_ + '/ranked?season=SEASON' + SEASON + '&api_key='+API_KEY
    info = requests.request("GET", url)
    body = info.json()
    headers = info.headers
    while (len(body)!=3):
        time_ten_seconds, time_ten_minutes = checkTime(headers, time_ten_seconds, time_ten_minutes)
        info = requests.request("GET", url)
        body = info.json()
        print(len(body))
        headers = info.headers
        if (len(body)==3):
            print("Breaking")
            break
    
    champions = body['champions']
    pref_champ = {}
    pref_champ_sessions = 0
    total_champ = {}
    discard = False
    for champ in champions:
        if champ['id'] == 0:
            if champ['stats']['totalSessionsPlayed']<MIN_MATCHES:
                discard = True
                break
            total_champs = champ 
        elif pref_champ_sessions < champ['stats']['totalSessionsPlayed']:
            pref_champ = champ
            pref_champ_sessions = champ['stats']['totalSessionsPlayed']
    if not discard:
        stats_json[id_]={'total': total_champs, 'pref_champ': pref_champ}    
    else:
        pass
        #print("Discarded")
    counter += 1
    time_ten_seconds, time_ten_minutes = checkTime(headers, time_ten_seconds, time_ten_minutes)

100 players done
200 players done
300 players done
400 players done
500 players done
600 players done
700 players done
800 players done
900 players done
1000 players done
1100 players done
1100 players done
1200 players done
1300 players done
1400 players done
1400 players done
1500 players done
1600 players done
1600 players done
1700 players done
1800 players done
1900 players done
2000 players done
2100 players done
2200 players done
2300 players done
2400 players done
2500 players done
2600 players done
2700 players done
2800 players done
2900 players done
3000 players done
3100 players done
3200 players done
3300 players done
3400 players done
3500 players done
3600 players done
3700 players done
3800 players done
3900 players done
4000 players done
4100 players done
4200 players done
4300 players done
4400 players done
4500 players done
4600 players done
4700 players done
4800 players done
4900 players done
5000 players done
5100 players done
5200 players done
5300 players done
5

In [167]:
print(len(stats_json))
with open('./summoner_stats.pkl', 'rb') as f:
    stats_json2 = pickle.load(f)
    print(len(stats_json2))



29309
29309


In [163]:
def getChampions():
    url = 'https://euw.api.pvp.net/api/lol/euw/v1.2/champion?api_key='+API_KEY
    info = requests.request("GET", url)
    return info.json()
def getChampionsPassive():
    url = 'https://global.api.pvp.net/api/lol/static-data/euw/v1.2/champion?dataById=true&champData=all&api_key='+API_KEY
    info = requests.request("GET", url)
    return info.json()


In [12]:
champs = getChampionsPassive()

In [18]:
champs['data']['1']

{'allytips': ['Storing a stun for use with her ultimate can turn the tide of a team fight.',
  'Striking killing blows on minions with Disintegrate enables Annie to farm extremely well early in the game.',
  "Molten Shield is a good spell to cast to work up to Annie's stun, so sometimes it's beneficial to grab at least 1 rank in it early."],
 'blurb': 'There have always been those within Noxus who did not agree with the evils perpetrated by the Noxian High Command. The High Command had just put down a coup attempt from the self-proclaimed Crown Prince Raschallion, and a crackdown on any form of ...',
 'enemytips': ["Annie's summoned bear, Tibbers, burns opposing units around himself. Try to keep your distance from him after he's been summoned.",
  'Summoner Smite can be used to help take down Tibbers.',
  "Keep an eye out for a white, swirling power around Annie. It means she's ready to unleash her stun."],
 'id': 1,
 'image': {'full': 'Annie.png',
  'group': 'champion',
  'h': 48,
  '

In [168]:
with open('./summoner_ids.pkl', 'rb') as f:
    summoner_ids = pickle.load(f)['ids']

In [169]:
def getELO(summoner_id):
    url = 'http://www.lolking.net/summoner/euw/' +  str(summoner_id)
    # We parse the response to analyze it 
    soup_project = BeautifulSoup(requests.request("GET", url).text, 'html.parser')
    elo = soup_project.findAll('div', {'class': 'summoner-name'})[0].findAll('div', {'class': 'medal-image'})
    if len(elo) > 1:
        elo = elo[1]['style'].split('medals/')[1][:-7]
    else:
        elo = ''
    return elo

In [89]:
with open('./summoner_elos.pkl', 'wb') as f:
    pickle.dump(elos, f)

In [181]:
with open('./summoner_elos.pkl', 'wb') as f:
    for s in range(len(summoner_ids)):
        if str(summoner_ids[s]) not in elos:
            elo = getELO(summoner_ids[s])
            elos[str(summoner_ids[s])] = elo
        if s%100==0 and s!=0:
            pickle.dump(elos, f)
            print(str(s), "players done")
with open('./summoner_elos.pkl', 'wb') as f:
    pickle.dump(elos, f)

100 players done
200 players done
300 players done
400 players done
500 players done
600 players done
700 players done
800 players done
900 players done
1000 players done
1100 players done
1200 players done
1300 players done
1400 players done
1500 players done
1600 players done
1700 players done
1800 players done
1900 players done
2000 players done
2100 players done
2200 players done
2300 players done
2400 players done
2500 players done
2600 players done
2700 players done
2800 players done
2900 players done
3000 players done
3100 players done
3200 players done
3300 players done
3400 players done
3500 players done
3600 players done
3700 players done
3800 players done
3900 players done
4000 players done
4100 players done
4200 players done
4300 players done
4400 players done
4500 players done
4600 players done
4700 players done
4800 players done
4900 players done
5000 players done
5100 players done
5200 players done
5300 players done
5400 players done
5500 players done
5600 players done
5

In [179]:
with open('./summoner_elos.pkl', 'wb') as f:
    pickle.dump(elos, f)

In [180]:
with open('./summoner_elos.pkl', 'rb') as f:
    print(len(pickle.load(f)))

25749


In [175]:
print(len(elos))

23900
